In [113]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
import pickle
from pydantic import BaseModel
import tiktoken
import os.path
import pprint

In [114]:
#https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

CHAT_GPT_MODEL = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model("gpt-4")

In [115]:
themes_cache = "cache/themes.pkl"

In [116]:
from os.path import expanduser
load_dotenv(os.path.join(expanduser("~"), ".env"))

True

In [117]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [118]:
data_file = "data_input.xlsx"
times = pd.read_excel(data_file, sheet_name="Data", index_col="ID")
demog = pd.read_excel(data_file, sheet_name="Demographic")

In [119]:
times.columns = [
    "start_time",
    "completion_time",
    "star_rating",
    "txt_what_liked",
    "txt_what_not_liked",
    "txt_do_to_improve",
    "txt_anything_else",
    "recommend_likelihood",
    "ref_num",
    "person_id",
    "start_group",
    "gender",
    "age_group",
    "country",
    "province",
    "number_finished",
    "reg_day",
    "reg_hour",
    "PPA",
    "has_result",
]

demog.columns = [
    "person_id",
    "start_group",
    "gender",
    "age_group",
    "country",
    "province",
    "number_finished",
    "reg_day",
    "reg_hour",
    "PPA",
    "has_result",
]

In [120]:
all_data = pd.merge(
    times, demog, on="person_id", how="left", suffixes=(None, "_demog")
).convert_dtypes()

all_data.drop(
    columns=[
        "start_group",
        "gender",
        "age_group",
        "country",
        "province",
        "number_finished",
        "reg_day",
        "reg_hour",
        "PPA",
        "has_result",
    ],
    inplace=True,
)

In [121]:
all_data = all_data.astype(
    {
        "start_group_demog": "category",
        "gender_demog": "category",
        "age_group_demog": "category",
        "country_demog": "category",
        "province_demog": "category",
    }
)

In [122]:
# Text cleanup
no_answer_text = "(none)"

txt_cols = [
    "txt_what_liked",
    "txt_what_not_liked",
    "txt_do_to_improve",
    "txt_anything_else",
]

all_data[txt_cols] = all_data[txt_cols].fillna(no_answer_text)

search_pattern = "|".join(["NA", "Na"])

for col in txt_cols:
    all_data[col] = all_data[col].str.replace(search_pattern, no_answer_text, regex=True)
    all_data[col] = all_data[col].replace("", no_answer_text)
    all_data[col] = all_data[col].str.replace(r"[^a-zA-Z ]", "", regex=True) #not needed
    all_data[col] = all_data[col].str.replace("\n", " ") #remove line breaks

In [123]:
# play with tiktoken
for col in txt_cols:
    test_text = " ".join(all_data[col].to_list())
    tokens = encoding.encode(test_text)
    print(col, len(tokens))

txt_what_liked 62476
txt_what_not_liked 96681
txt_do_to_improve 78861
txt_anything_else 58871


In [124]:
len_times = len(times)
lost_records = len(all_data) - len_times
lost_records_percent = lost_records / len_times
print(
    f"Data with no demographic records: {lost_records} rows, {lost_records_percent:.2%} of {len_times} total records."
)

Data with no demographic records: 55 rows, 0.95% of 5784 total records.


In [125]:
# GPT call for themes

def GPT_get_themes(responses):
    # OpenAI API call to summarize the text

    class Theme(BaseModel):
        theme_id: int
        theme_text: str

    class AllThemes(BaseModel):
        themes: list[Theme]

    messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"""The following is a list of responses to a single question in a market research survey. 
                
                Create an overall list of themes extracted from all answers. There shoud be at most 20 themes, 
                and they should have mninimal overlap. Each theme should be a maximum of 20 words.
                Each theme will have an index called theme_id and the theme itself as theme_text.
                Return all the themes ina list called 'themes'
                
                Here are the responses{responses}""",
            },
        ]

    completion   = client.beta.chat.completions.parse(
        model=CHAT_GPT_MODEL,
        temperature = 0.3,
        messages=messages,
        response_format=AllThemes,
    )

    # Extract the summary from the response
    return completion.choices[0].message.parsed
  

In [126]:
# GPT call for theme matching
# https://platform.openai.com/docs/guides/structured-outputs
def GPT_match_themes(themes, responses):

    class EachAnswer(BaseModel):
        response_id: int
        response_text: str
        theme_id: int
        theme_text: str

    class AllAnswers(BaseModel):
        classifications: list[EachAnswer]

    messages = [
            {
                "role": "system",
                "content": "You are an assistant for matching human responses to a survey to pre-existing themes.",
            },
            {
                "role": "user",
                "content": f"""I have a list of themes summarised over some responses to a survey question. The themes represent common topics found in the resposnes.
        Here are the themes: {themes}.
        
        I will give you the responses used to generate the themes. Each response has its own id called response_id.

        For each response, I want you to identify which one of the themes most closely represents the response.
        Return the answers in the object EachAnswer.
    
        Return the original response_id, the response_text, the theme_id and theme_text of the most representative theme.
        
        However, if the response text is {no_answer_text}, there will be no theme.
        In this case. return the original response_id, the response_text, 0 as the theme index and "no theme" as the theme text.

        Return all the EachAnswer objects in a final object called AllAnswers

        Here are the responses {responses}:""",
            },
        ]
    
    completion = client.beta.chat.completions.parse(
        model=CHAT_GPT_MODEL,
        temperature=0.1,
        messages=messages,
        response_format=AllAnswers,
    )

    return completion.choices[0].message.parsed

In [127]:
# GPT call to summarise inputs

def GPT_summarize_responses(inputs):
    # OpenAI API call to summarize the text
    response = client.chat.completions.create(
        model=CHAT_GPT_MODEL,  # Specify the model you want to use
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant for summarising survey responses.",
            },
            {
                "role": "user",
                "content": f"""You will be given a list of responses to a question in a survey. Your job is to extract key themes from the responses.
                Ignore any responses that are very short, are empty, or have the text {no_answer_text}
                Each theme should have a headline, followed by an explanatory paragraph. For each theme, provide from 1 to 3 verbatim quotes to illustrate the theme along side the explanatory paragraph.
                Don't provide any duplicated verbatim quotes.
                Sort the themes by their decreasing frequency of appearance. At the end, be sure to say which was the most commonly seen theme, and which was the least commonly seen.

                Here are your inputs:\n\n{inputs}""",
            },
        ],
    )

    # Extract the summary from the response
    summary = response.choices[0].message.content.strip()
    return summary

In [128]:
txt_cols_to_classify = [
    "txt_what_liked",
    "txt_what_not_liked",
    "txt_do_to_improve",
    "txt_anything_else",
]

txt_cols_to_summarise = [
    "txt_anything_else",
]

In [129]:
# Get the themes

def extract_themes(dataframe, columns):

    if os.path.isfile(themes_cache):

        with open(themes_cache, "rb") as picklefile:
            themes_dict = pickle.load(picklefile)

        return themes_dict

    themes_dict = {}

    for column in columns:

        print(f"Column: {column}")
        response_list = " ".join(dataframe[column].to_list())
        result = GPT_get_themes(response_list)
        theme_list = [(theme.theme_id, theme.theme_text) for theme in result.themes]
        themes_dict[column] = theme_list

    with open(themes_cache, "wb") as picklefile:
        pickle.dump(themes_dict, picklefile)

    return themes_dict

In [130]:
# Create themes
themes_dict = extract_themes(all_data, txt_cols_to_classify)

In [131]:
themes_dict

{'txt_what_liked': [(1, 'Well organized event with efficient logistics.'),
  (2, 'Spectacular scenic route around the Cape Peninsula.'),
  (3, 'Great atmosphere and camaraderie among participants.'),
  (4, 'Excellent support from spectators along the route.'),
  (5, 'Sufficient and well-placed water stations.'),
  (6, 'Smooth registration and number collection process.'),
  (7, 'Friendly and helpful marshals and volunteers.'),
  (8, 'Full road closures ensuring cyclist safety.'),
  (9, 'Positive energy and excitement from participants.'),
  (10, 'Improved timing system and tracking for participants.'),
  (11, 'Beautiful weather contributing to a great experience.'),
  (12, 'Community spirit and local support for the event.'),
  (13, 'Variety of cyclists participating, from novices to pros.'),
  (14, 'Festive atmosphere at the start and finish.'),
  (15, 'Challenging yet enjoyable course for all levels.'),
  (16, 'Great post-race celebrations and hospitality.'),
  (17, 'Increased public

In [144]:
# Classify the columns

def classify_text_column(orig_dataframe, columns):

    # Create a deep copy with indices to return for pickling
    dataframe = orig_dataframe.copy()

    with open(themes_cache, "rb") as picklefile:
        themes_dict = pickle.load(picklefile)

    for column in columns:

        print(f"Column: {column}")
        response_list = dataframe[column].to_list()
        expected_num_outputs = len(response_list)

        responses_input = [
            f"{idx} {txt}"
            for idx, txt in zip(dataframe[column].index, dataframe[column])
        ]
        # print(responses_input)

        expected_num_outputs = len(response_list)

        # responses_input = "\n".join(response_list)
        responses_input_for_output = "".join([f"{i}. {response} \n" for i, response in enumerate(response_list)])

        # responses_input = str(response_list)

        with open(f"outputs/A {column} responses_input.txt", "w") as text_file:
            text_file.write(responses_input_for_output)

        themes_for_input = themes_dict[column]

        # print(themes_for_input)

        response = GPT_match_themes(themes_for_input, responses_input)

        actual_num_outputs = len(response.classifications)

        #pprint.pp(response.classifications)

        print(
            f"Expected, actual responses: {expected_num_outputs}, {actual_num_outputs}"
        )

        with open(f"outputs/B {column} classifications.txt", "w") as text_file:
            for el in response.classifications:
                text_file.write(f"{el.response_id} {el.response_text} {el.theme_id} {el.theme_text} \n")

        classified_themes = [resp.theme_text for resp in response.classifications]

        column_index = dataframe.columns.get_loc(column)
        new_name = f'{column}_theme'

        try:
            dataframe.insert(column_index + 1, new_name, classified_themes)
            print(f'New column {new_name} inserted')
            # print(dataframe)

        except:
            print("failed to insert column")
            pass

        # dataframe.to_pickle(classifications_cache)

    return dataframe

In [145]:
# Loop through chunks

working_data = all_data[0:23].copy()
total_records = len(working_data)

print("Total Records", total_records)
chunk_size = 5
working_txt_cols_to_classify = txt_cols_to_classify#[0:1]
chunks_folder = 'df_chunks/'

start, end = 0, chunk_size

while end <= total_records:

    print('Start, end = ', start , end)

    current_chunk = working_data[start:end].copy()

    classifications = classify_text_column(
        current_chunk,
        working_txt_cols_to_classify,
    )

    # print(classifications['txt_what_liked'])

    output_file = f"{chunks_folder}df {start:04d}-{end:04d}.pkl"

    with open(output_file, "wb") as f:
        classifications.to_pickle(f)

    if start + chunk_size <= total_records:
        start = end + 1
        end = start + chunk_size

    else:
        remaining_records = total_records - end
        start = end + 1
        end = start + remaining_records

    print("New start, end", start, end)

    print()

# classifications
# working_data

Total Records 23
Start, end =  0 5
Column: txt_what_liked
Expected, actual responses: 5, 5
New column txt_what_liked_theme inserted
Column: txt_what_not_liked
Expected, actual responses: 5, 5
New column txt_what_not_liked_theme inserted
Column: txt_do_to_improve
Expected, actual responses: 5, 5
New column txt_do_to_improve_theme inserted
Column: txt_anything_else
Expected, actual responses: 5, 5
New column txt_anything_else_theme inserted
New start, end 6 11

Start, end =  6 11
Column: txt_what_liked
Expected, actual responses: 5, 5
New column txt_what_liked_theme inserted
Column: txt_what_not_liked
Expected, actual responses: 5, 5
New column txt_what_not_liked_theme inserted
Column: txt_do_to_improve
Expected, actual responses: 5, 4
failed to insert column
Column: txt_anything_else
Expected, actual responses: 5, 5
New column txt_anything_else_theme inserted
New start, end 12 17

Start, end =  12 17
Column: txt_what_liked
Expected, actual responses: 5, 5
New column txt_what_liked_theme

In [146]:
chunks_folder = "df_chunks/"

df_chunks = []

print('Resonsituting...')
for filename in os.listdir(chunks_folder):
    print(filename)
    temp_df = pd.read_pickle(f'{chunks_folder}{filename}')
    df_chunks.append(temp_df)

reconstitued = pd.concat(df_chunks)

reconstitued.to_pickle('outputs/final_reconsituted.pkl')
reconstitued

Resonsituting...
df 0000-0005.pkl
df 0018-0023.pkl
df 0006-0011.pkl
df 0012-0017.pkl


In [147]:
reconstitued

,start_time,completion_time,star_rating,txt_what_liked,txt_what_liked_theme,txt_what_not_liked,txt_what_not_liked_theme,txt_do_to_improve,txt_do_to_improve_theme,txt_anything_else,...,start_group_demog,gender_demog,age_group_demog,country_demog,province_demog,number_finished_demog,reg_day_demog,reg_hour_demog,PPA_demog,has_result_demog
0,2024-03-20 13:45:57,2024-03-20 13:47:45,<NA>,none,no theme,none,no theme,none,no theme,none,...,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2024-03-20 14:00:18,2024-03-20 14:02:21,5,The Team who organise it,Impressive organization for such a large event.,Nothing comes to mind,no theme,Alp du Hez experience at the top of Alp du Sui...,Enhance the expo experience with more cycling-...,brilliant,...,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2024-03-20 14:27:17,2024-03-20 14:28:30,4,well organised great helpers,Friendly and helpful marshals and volunteers.,start a little too late,"Poor organization at the start, leading to con...",start earlier,Implement a rolling start to reduce bottleneck...,much better than KMs,...,9E,2,45-49,South Africa,Western Cape,3,8,11,<NA>,1
3,2024-03-20 14:27:17,2024-03-20 14:28:48,5,Road closures,Full road closures ensuring cyclist safety.,It is getting dangerous with larger groups,Large start groups causing safety concerns.,Rider safety in large groups,Improve communication about race rules and eti...,Nope,...,#,2,40-44,South Africa,Western Cape,20,8,13,<NA>,1
4,2024-03-20 14:27:32,2024-03-20 14:28:52,5,Well organized event and great route,Well organized event with efficient logistics.,Think its starting to get quite pricey,High entry fees compared to value received.,Na,no theme,Na,...,8B,2,35-39,South Africa,Western Cape,3,7,12,<NA>,1
18,2024-03-20 14:27:29,2024-03-20 14:30:42,3,Route people vibe,Great atmosphere and camaraderie among partici...,The start chute loading procedure There are wa...,"Poor organization at the start, leading to con...",The start chute procedure,Improve the organization of the starting area ...,none,...,2E,2,45-49,South Africa,Western Cape,19,7,17,<NA>,1
19,2024-03-20 14:27:30,2024-03-20 14:30:44,4,The Vibe,Positive energy and excitement from participants.,The start location reeked of human excrement,Unpleasant smell from homeless encampments at ...,Move the start or clean it up thoroughly,Improve the organization of the starting area ...,Overall satisfied with organizing but smaller ...,...,@,2,40-44,South Africa,Western Cape,21,9,13,<NA>,1
20,2024-03-20 14:29:38,2024-03-20 14:30:45,4,All round great event,Impressive organization for such a large event.,Groups were extremely large especially racing ...,Large start groups causing safety concerns.,Smaller bunches for racing groups,Implement a rolling start to reduce bottleneck...,Better goodie bag,...,&,2,35-39,South Africa,Western Cape,12,7,12,<NA>,1
21,2024-03-20 14:28:50,2024-03-20 14:30:46,3,Overall enjoyed it,Overall memorable experience with beautiful vi...,Cut offs too earley for late starters and cons...,Windy conditions affecting race experience.,New way of cutoff times,Improve communication about race rules and eti...,none,...,<NA>,2,45-49,South Africa,Western Cape,7,8,12,<NA>,<NA>
22,2024-03-20 14:28:32,2024-03-20 14:30:48,5,Much bigger and better experience than the fir...,Impressive organization for such a large event.,none,no theme,Watch the cost People are under huge financia...,no theme,none,...,2A,2,60-64,South Africa,Kwazulu-Natal,9,8,11,<NA>,1


In [ ]:
themes_dict